# RSA Implementation and common mistakes

## How to set-up RSA for Dummies
The first problem we have is generating primes of sufficient size. Let's start by making a little function to check if big numbers are prime

In [21]:
from random import randint
smallPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 
            127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 
            257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 
            401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

def isPrime(n):
    # Quickly test small factors.
    if n in smallPrimes:
            return True

    # Use Fermat's little theorem to do the harder work on larger primes. (https://en.wikipedia.org/wiki/Fermat%27s_little_theorem)
    for i in range(0, 50): 
        base = randint(2, n-1)
        if pow(base, n-1, n) != 1: 
            return False
    
    return True

And let's check we've not been completely stupid by checking a few cases:

In [22]:
print(2, isPrime(2))
print(3, isPrime(3))
print(10, isPrime(10))

print(65537, isPrime(65537))
print(65539, isPrime(65539))
print(65541, isPrime(65541))



2 True
3 True
10 False
65537 True
65539 True
65541 False


And now for something a bit bigger. 

This prime was nicked from the [RFC 3526](https://www.rfc-editor.org/rfc/rfc3526.txt). 

In [23]:
bigPrime = """FFFFFFFF FFFFFFFF C90FDAA2 2168C234 C4C6628B 80DC1CD1
      29024E08 8A67CC74 020BBEA6 3B139B22 514A0879 8E3404DD
      EF9519B3 CD3A431B 302B0A6D F25F1437 4FE1356D 6D51C245
      E485B576 625E7EC6 F44C42E9 A637ED6B 0BFF5CB6 F406B7ED
      EE386BFB 5A899FA5 AE9F2411 7C4B1FE6 49286651 ECE45B3D
      C2007CB8 A163BF05 98DA4836 1C55D39A 69163FA8 FD24CF5F
      83655D23 DCA3AD96 1C62F356 208552BB 9ED52907 7096966D
      670C354E 4ABC9804 F1746C08 CA18217C 32905E46 2E36CE3B
      E39E772C 180E8603 9B2783A2 EC07A28F B5C55DF0 6F4C52C9
      DE2BCBF6 95581718 3995497C EA956AE5 15D22618 98FA0510
      15728E5A 8AACAA68 FFFFFFFF FFFFFFFF"""

bigPrime = bigPrime.replace(" ", "")
bigPrime = bigPrime.replace("\n", "")

bigDaddy = int(bigPrime, 16)

isPrime(bigDaddy)

True

Alright, so how do we generate a large random prime. The code below implements the process:

In [24]:
def bigPrime(bits):
    from random import getrandbits
    p = getrandbits(bits)
    while not isPrime(p):
        p = getrandbits(bits)
    return p

Does this prove that the numbers are prime? Of course not, they're sort of "industrial strength" primes.

Alrighty then, let's generate some primes:

In [25]:
p = bigPrime(1024)
q = bigPrime(1024)

p,q

(154933371817119426610152223125278048863282712520104689292970954205868662770683601612898119720528567669187238603901901257438253158561193242785718441496891020893566166001004277450782112380915453751003783105310600856039822274815678293543894337638723938631113893113200934652247939811675948458681828401299728237797,
 43676229164615742854226774478725647612517491379615006993947215446666128738942122506904021685570699320326733873812999868363374506352373316357445199700180962345816865315946221935965338414012719312959138139876956436247232725749543268214345756672967627831643473294566409226419399077385634277856398840610540492559)

And multiply to get a modulus:

In [26]:
n = p * q
n

6766905452731126291558939402574517219404683039566540168997515764362789655562919407677294320497211011793152647734430889239104853154812949233517055137345974034662919993397384038693672799921251298683990543375126598899600763922299536389792370941148601450576098032716899726680458913643741844276119247513644675168744410417626890414848616292478087783331901343468725396255168793766795057968242864856416373311440574788046298877858267754411732384490010666426301667126824498073440479342878101830247027466408016026787451751350289910040528306926568790704463672444281671780665003661194872106709915857572294925846549577421461052523

# Explanation of the role of Extended GCD / LCM

The extended Euclian algorithm is the standard GCD algorithm extended to show more of its internal state. It's used to find solutions to the equations of the form:

$$(ax + by) = gcd(x, y)$$

In our case, we're looking for a solution such that:

$$e \times d \equiv {1 \mod {\text{lcm} (p-1,q-1)}}$$

So, if we substitute:

$$ex + (b \times {\text{lcm} (p-1,q-1)}) = gcd(e, {\text{lcm} (p-1,q-1)})$$

But by requirement for the scheme to work:

$$gcd(e, {\text{lcm} (p-1,q-1)}) = 1$$

So we can rewrite the right hand side to 1:

$$(e \times x) + (b \times {\text{lcm} (p-1,q-1)}) = 1$$

Now we take both sides  $\mod {\text{lcm} (p-1,q-1)}$:

$$((e \times x) + (b \times {\text{lcm} (p-1,q-1)})) \mod {\text{lcm} (p-1,q-1)} \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

But notice that we just have $b$ multiplied by the thing in the mod, so we can simplify:

$$ex \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

This looks like our target equation above, so $x=d$ which is our encryption exponent.

In [27]:
def gcdExtended(a, b):
		if a == 0 :   
			return b,0,1
		     
		gcd,x1,y1 = gcdExtended(b % a, a)  
     
		x = y1 - (b//a) * x1  
		y = x1  
	     
		return gcd,x,y 

def lcm(a, b):
	gcd, _, _ = gcdExtended(a, b)
	return (a * b) // gcd


Right, now we get to the business end of setting up RSA. First we need to compute phi(n), which is the number of numbers relatively prime to "n"

In [28]:
phi_n = lcm(p-1, q-1)

Next we need to define what we want for the encryption exponent. It needs to be big enough so that when we do m^e it is bigger than p and thus "loops around"

In [29]:
e = 65537

Now the final step, let's compute d from phi_n and e.

In [30]:
_, d, _ = gcdExtended(e, phi_n)

if d < 0:
	d = phi_n + d # d is negative. Lost a good 45 minutes there.

d

295304173135954059445177025060090013999685574456571171755388484155173084134305041517876340946671704437621749126760033923185984711274013683993145516163533358852799963091330368351677742462651307112565618720003388511113694322562471185357983747984878773038858055351485927312908929200621048791212613453301551352396981710111923707246271856559498916465636957126010594044725830980465757695220181309896954441754675073680553786478020485494070517386951342704575655799915958808389219850682102369476918218417571003763952534409957967053342900288286816222686412766211570581644198583163750716880567772860226083913236826213619940513

Alright, let's check everything works!!!

In [31]:
m = 420

cipherText = pow(m, e, n)
plainText = pow(cipherText, d, n)

m, e, plainText

(420, 65537, 420)

# Now let's put the whole thing in to one neat package

In [32]:
class RSAFixedKey:
    def __init__(self, e=65537):
        self.p = bigPrime(1024)
        self.q = bigPrime(1024)
        self.n = self.p * self.q
        self.e = e
        phi_n = lcm((self.p-1),(self.q-1))
        _, self.d, _ = gcdExtended(e, phi_n)
        if self.d < 0:
            self.d = phi_n + self.d

    def encrypt(self, data):
        return pow(data, self.e, self.n)

    def decrypt(self, data):
        return pow(data, self.d, self.n)

And let's test it out!

In [33]:
rsa = RSAFixedKey()

plainTextBefore = 69
cipherText=rsa.encrypt(69)
plainTextAfter=rsa.decrypt(cipherText)

plainTextBefore, cipherText, plainTextAfter


(69,
 798738554215779836127140703188084246354852797625464895619267944830104714044857918349370480956428122396474160795489106197107180703678480685212305954560461513197439662064764088963500576085624676560312705529613695120508366004062940181682694063601773722272223593171930265174627259065824590220214695913577940476425152520738603547896437336835588727386042905543981088154159217150561745559141365331779564765124996437074204015444530548052433431047059577958302493018418370035430366503574206554162229075558571463413469594678223305525981474706233612219168295667725568871243568376736239078866640865079831037703176156488239961627,
 69)

# Attack: Not picking a big enough exponent for $e$

In [43]:
rsa = RSAFixedKey(e=3)

In [44]:
from SemanticSecurityGame import SemanticSecurityGame

class RSAGameStrategyLowExponent:
    def getMessages(self, trialNumber):
           m0 = 3
           m1 = 2
           
           return m0, m1

    def challenge(self, challenge, trialNumber):
           result = challenge ** (1/3)

           if result == 3:
                return 0 
           else:
                return 1

strategy = RSAGameStrategyLowExponent()
game = SemanticSecurityGame(rsa, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 50}

# Attack: The cipher-text is the same each time!

In [45]:
rsa = RSAFixedKey()

In [37]:
from SemanticSecurityGame import SemanticSecurityGame

class RSAGameStrategy:
    def getMessages(self, trialNumber):
            if trialNumber == 0:
                m0 = 69
                m1 = 69
            else:
                m0 = 69
                m1 = 420
            return m0, m1 

    def challenge(self, challenge, trialNumber):
            if (trialNumber == 0):
                self.savedMessage = challenge
            if self.savedMessage == challenge:
                return 0
            else:
                return 1

strategy = RSAGameStrategy()
            
game = SemanticSecurityGame(rsa, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 49}

# Attack: Algebraic properties of RSA

In [38]:
m_0 = 69
m_1 = 420

m_2 = 420*69
m_2

28980

And now we find the cipher-text for $c_2$ from the cipher-texts computed from $m_0$ and $m_1$.

In [39]:
c_0 = rsa.encrypt(m_0)
c_1 = rsa.encrypt(m_1)

c_2 = c_0*c_1 % rsa.n



The algebra around what is happening: 

$$c_2 = m_0^{e}m_1^{e} \mod n$$
$$c_2 = (m_0 \times m_1)^{e} \mod n$$

And now we decrypt:

In [40]:
rsa.decrypt(c_2)

28980

Why do we get the same answer as $m_2$?

Because:

$$ m_2 = m_0 \times m_{1} $$

But running the decryption operation on $c_2$ give us:

$$c_2^{d} \mod n \equiv (m_0 \times m_1)^{ed} \mod n \equiv (m_0 \times m_1) \mod n$$

# How to use RSA properly

https://en.wikipedia.org/wiki/Optimal_asymmetric_encryption_padding